机器学习-毕业设计

目标：根据 Loan Club 数据集构建衍生变量并运用 stacking 提升效果。
要求：
1. 请根据之前的作业的baseline给出未生成衍生变量的准确率；
2. 记录构建衍生变量（和进行变量选择的过程），可记录（但不要求）探索性数据分析结果、变量选择过程和衍生变量构建过程等；
3. 请注意在构建衍生变量或进行变量选择之后，需要进一步进行调参；
4. 请进行 stacking 并比较结果；
5. 最终请提交报告和测试集预测结果。

In [1]:

!git clone https://github.com/aiblogs/machineLearningDataSet.git


Cloning into 'machineLearningDataSet'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 7 (delta 1), reused 4 (delta 1), pack-reused 0
Unpacking objects: 100% (7/7), done.


In [12]:
!pip install lightgbm xgboost catboost category-encoders sklearn pandas==1.1.5

     |████████████████████████████████| 67.4 MB 28 kB/s 
     |████████████████████████████████| 80 kB 9.6 MB/s 


In [14]:

import pandas as pd  
import numpy as np  
#import pickle
import lightgbm as lgb  
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score  
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import StratifiedKFold



In [16]:

train = pd.read_csv('./machineLearningDataSet/train_final.csv', engine='python')
test = pd.read_csv('./machineLearningDataSet/test_final.csv', engine='python')

train_x = np.array(train.drop(['loan_status'],1))
#print(train_x)
train_y = train.iloc[:,15]
#print(train_y)
test_x = np.array(test.drop(['loan_status'],1))
#print(test_x)
test_y = test.iloc[:,15]
#print(test_y)

In [15]:
def stacking(model, x_train, y_train, x_test, n_splits):
    n_train, n_test = x_train.shape[0], x_test.shape[0]
    kf = StratifiedKFold(n_splits=n_splits, random_state=0)
    oof_train = np.empty((n_train, ))
    oof_test = np.empty((n_test, ))
    oof_test_skf = np.empty((n_splits, n_test))
    for i, (train_index, test_index) in enumerate(kf.split(x_train, y_train)):
        kf_x_train = x_train[train_index]
        kf_y_train = y_train[train_index]
        kf_x_test = x_train[test_index]
        model.fit(kf_x_train, kf_y_train)
        oof_train[test_index] = model.predict(kf_x_test)
        oof_test_skf[i, :] = model.predict(x_test)
    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

In [6]:
# create dataset for lightgbm  
lgb_train = lgb.Dataset(train_x, train_y)  
lgb_test = lgb.Dataset(test_x, test_y, reference=lgb_train)  

In [18]:
# specify configurations as a dict 
#for val in [2, 4, 8, 16, 32, 64, 128]:
params = {'boosting_type': 'gbdt',
      'num_thread': 4, 
      'num_leaves': 16, 
      'metric': 'binary', 
      'objective': 'binary',
      'num_round': 4000, 
      'learning_rate': 0.02,
      'feature_fraction': 0.8, 
      'bagging_fraction': 0.8,
      'max_depth': 8}


In [19]:
# train  
print('Start training...')  
gbm = lgb.train(params,lgb_train,num_boost_round=10000,valid_sets=lgb_test,early_stopping_rounds=1000)  
print('Finished training...')  

Start training...
[1]	valid_0's binary_logloss: 0.482401
Training until validation scores don't improve for 1000 rounds.
[2]	valid_0's binary_logloss: 0.471303
[3]	valid_0's binary_logloss: 0.460918
[4]	valid_0's binary_logloss: 0.451144
[5]	valid_0's binary_logloss: 0.441934
[6]	valid_0's binary_logloss: 0.433229
[7]	valid_0's binary_logloss: 0.424993
[8]	valid_0's binary_logloss: 0.41716
[9]	valid_0's binary_logloss: 0.409708
[10]	valid_0's binary_logloss: 0.402606
[11]	valid_0's binary_logloss: 0.395843
[12]	valid_0's binary_logloss: 0.389375


/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[13]	valid_0's binary_logloss: 0.383193
[14]	valid_0's binary_logloss: 0.377255
[15]	valid_0's binary_logloss: 0.371573
[16]	valid_0's binary_logloss: 0.366189
[17]	valid_0's binary_logloss: 0.361029
[18]	valid_0's binary_logloss: 0.355993
[19]	valid_0's binary_logloss: 0.351159
[20]	valid_0's binary_logloss: 0.346499
[21]	valid_0's binary_logloss: 0.342016
[22]	valid_0's binary_logloss: 0.337687
[23]	valid_0's binary_logloss: 0.333511
[24]	valid_0's binary_logloss: 0.32949
[25]	valid_0's binary_logloss: 0.325611
[26]	valid_0's binary_logloss: 0.321866
[27]	valid_0's binary_logloss: 0.318257
[28]	valid_0's binary_logloss: 0.314762
[29]	valid_0's binary_logloss: 0.31139
[30]	valid_0's binary_logloss: 0.308131
[31]	valid_0's binary_logloss: 0.304982
[32]	valid_0's binary_logloss: 0.301969
[33]	valid_0's binary_logloss: 0.299011
[34]	valid_0's binary_logloss: 0.296144
[35]	valid_0's binary_logloss: 0.293411
[36]	valid_0's binary_logloss: 0.290711
[37]	valid_0's binary_logloss: 0.288102
[3

In [20]:
preds = gbm.predict(test_x, num_iteration=gbm.best_iteration) 
size = np.size(preds)
print(size)

50000


In [21]:
threshold = 0.5  
for i in range(0,size):
    if preds[i] >= threshold:
       preds[i] = 1
    else:  
       preds[i] = 0

In [22]:
accuracy = accuracy_score(test_y,preds) 
print(accuracy)

0.91776


**调参过程**
#1
params = {'boosting_type': 'gbdt',
      'num_thread': 4, 
      'num_leaves': 16, 
      'metric': 'binary', 
      'objective': 'binary',
      'num_round': 4000, 
      'learning_rate': 0.02,
      'feature_fraction': 0.8, 
      'bagging_fraction': 0.8,
      'max_depth': 8}
0.91776